In [1]:
import urllib.request
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
import pandas as pd

from selenium import webdriver
from time import sleep
import json
from math import radians, cos, sin, asin, sqrt
import copy

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing

import folium

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [64]:
def distance(x_lat, x_long, y_lat, y_long):
    """
    Расстояние в километрах между думя точками
    """
    # конвертация в радианы
    x_lat, x_long, y_lat, y_long = map(radians, [x_lat, x_long, y_lat, y_long])

    dlong = y_long - x_long 
    dlat = y_lat - x_lat 
    a = sin(dlat/2)**2 + cos(x_lat) * cos(y_lat) * sin(dlong/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Радиус Земли в км - 6371
    km = 6371* c
    return km

# Парсер открытой информации из HeadHanter

In [2]:
def get_html(url):
    response = urllib.request.urlopen(url)
    return response.read()

In [3]:
def get_client_urls(url):
    client_urls = list()
    html = get_html(url)
    soup = BeautifulSoup(html)
    div_urls = soup.find_all('div', itemscope = 'itemscope')
    for div in div_urls:
        client_urls.append('https://hh.ru/resume/' + div['data-hh-resume-hash'])
    return client_urls

In [7]:
class Client:
    def __init__(self, url):
        html = get_html(url)
        soup = BeautifulSoup(html)
        self.gender = soup.find('span', itemprop='gender').text
        try:
            birthday_str = soup.find('meta', itemprop='birthDate')['content']
            self.birthday = datetime.strptime(birthday_str, "%Y-%m-%d").date()
        except TypeError:
            self.birthday = None

        # формат строки возвращаемый объектом soup u'65\xa0000 руб.' 
        try:
            self.salary = int(soup.find('span', class_='resume-block__salary').text.replace(u'\xa0', u'')[:-5])
        except ValueError: 
            self.salary = None
            
        self.spec = soup.find('span', class_='resume-block__title-text').text 
        self.fio = 'ФИО'
        
    def __repr__(self):
        return '{} {} {} "{}" {} р.'.format(self.fio, self.gender,
                                      self.birthday, self.spec, self.salary)

In [17]:
def parse(area, page_count):
    result = list()
    for page in range(1,page_count+1):
        search_page_url = ('https://hh.ru/search/resume?exp_period=all_time&order_by=relevance&no_magic=true&area=1'
                        '&text=&pos=full_text&logic=normal&clusters=true&page={}').format(area, page)
        client_urls = get_client_urls(search_page_url)
        for url in client_urls: 
            result.append(Client(url)) 
    return result 

In [18]:
clients = parse(1, 1) # area = 1 - Moscow; page_count = 1

In [19]:
for client in clients: print(client)

ФИО Женщина 1978-10-03 "Менеджер по работе с ключевыми клиентами" 80000 р.
ФИО Женщина 1990-09-15 "Региональный менеджер" 40000 р.
ФИО Женщина 1977-11-21 "Главный  бухгалтер" None р.
ФИО Мужчина 1971-11-13 "Коммерческий директор, директор по продажам, руководитель отдела продаж" 150000 р.
ФИО Male 1989-05-31 "Mechanical Engineer (Power and Energy)" None р.
ФИО Женщина 1986-01-28 "SMM-менеджер" 30000 р.
ФИО Женщина 1989-03-01 "Бухгалтер" None р.
ФИО Мужчина 1993-11-29 "Специалист по оценке персонала" None р.
ФИО Мужчина 1975-08-23 "Заместитель генерального директора / генеральный директор" None р.
ФИО Мужчина 1959-12-22 "Инженер-электрик, прораб." 70000 р.
ФИО Мужчина 1980-09-18 "Личный водитель" 50000 р.
ФИО Женщина 1985-09-06 "Менеджер проекта, Директор по маркетингу." 70000 р.
ФИО Мужчина 1986-04-24 "Специалист отдела продаж" 60000 р.
ФИО Женщина 1996-09-26 "Начинающий специалист" None р.
ФИО Женщина 1976-05-12 "Преподаватель, помощник преподавателя английского языка, репетитор" None

# Парсинг телефонов собственников продающих жильё c сайта Yandex.Недвижимость.

In [3]:
class FlatOwner:
    "Объект хозяин квартиры"
    
    def __init__(self, name, telephone, lat, long, neighbours, price, info):
        self.name = name
        self.telephone = telephone
        self.lat = lat # широта объекта недвижимости
        self.long = long # долгота объекта недвижимости
        self.neighbours = neighbours or list() # ближайшие отделения
        self.price = price or None # цена объекта недвижимости
        self.info = info or list() # площадь, жилая площадь, кухня, этаж, год постройки
        
    def __repr__(self):
        return '{} {} {} {}'.format(self.name, self.telephone,
                                            self.lat, self.long)    

In [4]:
class YandexParser:
    
    def __init__(self):
        self.driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')
        
    def get_searh_page(self, url):
        "Функция возвращает список url адресов объявлений со страницы поисковой выдачи"
        
        self.driver.get(url)
        try:
            try:
                elements = driver.find_elements_by_xpath("//h3[@class='serp-item__head serp-item__row']/a")
            except:
                elements = self.driver.find_elements_by_xpath("//div[@class='OffersSerpItem__generalInfo']/a")
        except: 
            elements = self.driver.find_elements_by_xpath("//li[@class='OffersSerpItem OffersSerpItem_mini OffersSerp__list-item OffersSerp__list-item_type_offer']/a")
        result = list()
        for element in elements: result.append(element.get_attribute('href'))
            
        return result

    def parse_owner(self, url):
        "Функция возвращает объект FlatOwner создавая его используя странцу объявления"
        
        self.driver.get(url)
        try:
            name = self.driver.find_element_by_css_selector('.offer-card__author-name.ellipsis').text
        except:
            name = None
        try:
            location = self.driver.find_element_by_id('offer-map')   
            location_dict = json.loads(location.get_attribute('data-bem'))['offer-map']['placemarks'][0]

            lat = location_dict['lat']
            long = location_dict['lon'] 
        except:
            lat, long = 0, 0
        
        try:
            price = self.driver.find_element_by_xpath("//h3[@class='offer-price offer-card__price offer-card__price']").text
        except:
            price = None
            
        try:
            elements = self.driver.find_elements_by_xpath("//div[@class='offer-card__main-feature-title']")
        except:
            elements = None
        
        info = list()
        for element in elements:
            info.append(element.text)
            
        button = self.driver.find_element_by_xpath("//div[@class='offer-card__contacts-phones-timetable']/span/button")
        button.click()
        sleep(0.5)
        try:
            telephone = self.driver.find_element_by_css_selector('.helpful-info__contact-phones-string').text
        except: 
            telephone = None
        
        return FlatOwner(name, telephone, lat, long, None, price, info)
    
    def parse(self, start, end):
        result = list()
        pattern = 'https://realty.yandex.ru/moskva/kupit/kvartira/?newFlat=NO&agents=NO&page='
        
        for i in range(start, end+1): 
            url = pattern + str(i)
            page_results = list()
            for url in self.get_searh_page(url): 
                page_results.append(self.parse_owner(url))
            pickle.dump(page_results, open('page_result_{}.data'.format(i),'wb'))
            result.extend(page_results)
            print('Страница {}'.format(i))
            
        self.driver.quit()
        return copy.deepcopy(result)
        

In [9]:
yandex_parser = YandexParser()

In [10]:
%time flat_owner_result = yandex_parser.parse(1,24)

Страница 1
Страница 2
Страница 3
Страница 4
Страница 5
Страница 6
Страница 7
Страница 8
Страница 9
Страница 10
Страница 11
Страница 12
Страница 13
Страница 14
Страница 15
Страница 16
Страница 17
Страница 18
Страница 19
Страница 20
Страница 21
Страница 22
Страница 23
Wall time: 57min 23s


In [5]:
def wrapper(start, end):
    result = YandexParser().parse(start, end)
    return result
    

In [6]:
def parallel_parsing(page_count, workers=2):
    params = zip(range(1, page_count+1, workers), range(workers+1, page_count+1, workers))
    futures = set()
    with ThreadPoolExecutor(max_workers = workers) as executor:
        for start, end in params:
            future = executor.submit(wrapper, start, end)
            print('{} парсит {}-{}'.format(future, start, end-1))
            futures.add(future)

        result = []
        for future in concurrent.futures.as_completed(futures):
            err = future.exception()
            if err is None:
                print(future, ' Пропарсено: ', len(future.result()))
                result.extend(future.result())
            else: 
                raise err
    return result

In [12]:
pickle.dump(flat_owner_result, open('flat_owners_v1.data', 'wb'))

In [5]:
flat_owner_result = pickle.load( open('flat_owners.data', 'rb'))

# Парсинг отделений Росбанка

In [6]:
Rosbank = namedtuple('Rosbank', 'adress telephone lat long')

In [33]:
driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')
url = 'https://yandex.ru/maps/213/moscow/?source=serp_navig&ll=37.629112%2C55.761047&z=11&mode=search&text=Росбанк%2C%20отделения&sll=37.629112%2C55.761047&sspn=0.157242%2C0.245461&sctx=ZAAAAAgBEAAaKAoSCVx%2B2LmG0EJAES9bof5p4UtAEhIJAAAAAADo5z8Rg1Z%2BHkVrzz8iBAABAwYoBTABOKLErLSj08qGJEDVAUgBVc3MzD5YAGIcdHlwZT1iaXosZ2VvLHRyYW5zaXQsb2JqZWN0c2oCcnVwAJ0BzczMPaABAKgBAA%3D%3D'
driver.get(url)

In [39]:
def get_pages():
    page_list = list()
    elements = driver.find_elements_by_xpath("//a[@class='search-business-snippet-view__title']")
    for element in elements: page_list.append(element.get_attribute('href'))
    return page_list

In [40]:
page_list = get_pages()

In [41]:
len(page_list)

55

In [42]:
pickle.dump(page_list,open('rosbank_division_pages.data','wb'))

In [45]:
def get_divisions():
    result = list()
    for page in page_list: 
        driver.get(page)
        try:
            telephone = driver.find_element_by_xpath("//div[@class='org__phone-number']").text
        except:
            telephone = None
        
        adress = driver.find_element_by_xpath("//a[@class='org__address-name']").text
        location = str(driver.find_element_by_xpath("//a[@class='org__address-name']").get_attribute('href')).split('/')[-2] 
        long, lat = map(float, location.split(','))
        result.append(Rosbank(adress, telephone, lat, long))
        
    return result 

In [46]:
result = get_divisions()

In [47]:
pickle.dump(result, open('Rosbank_divisions_v1.data','wb'))

In [7]:
rosbank_divisions = pickle.load(open('Rosbank_divisions_v1.data','rb'))

# Поиск ближайших к объекту недвижимости отделений Росбанка

In [8]:
Neighbour = namedtuple('Neighbour', 'rosbank distance')

In [65]:
for owner in flat_owner_result:  
    neighbours = list()
    for division in rosbank_divisions:
        neighbours.append(Neighbour(division, distance(owner.lat, owner.long, division.lat, division.long)))
    neighbours = sorted(neighbours, key = lambda x: x.distance)[:3]
    owner.neighbours =  copy.deepcopy(neighbours)   

In [77]:
flat_owner_result = [x for x in flat_owner_result if x.telephone is not None]

In [78]:
len(flat_owner_result)

275

In [81]:
for owner in flat_owner_result[3::20]:  
    print(owner, owner.neighbours[:1],end='\n\n')

Александр +7 916 060-85-42 55.76364 37.82838 [Neighbour(rosbank=Rosbank(adress='Мартеновская ул., 11/12, Москва', telephone='8 (800) 200-54-34', lat=55.756072, long=37.804781), distance=1.6994546034346618)]

Елена +7 903 013-39-15, +7 925 133-62-07 55.628704 37.73951 [Neighbour(rosbank=Rosbank(adress='Люблинская ул., 165, корп. 1, Москва', telephone='8 (800) 200-54-34', lat=55.650063, long=37.745924), distance=2.4088827912458974)]

Светлана +7 916 710-41-64 55.567493 37.47871 [Neighbour(rosbank=Rosbank(adress='бул. Дмитрия Донского, 2, корп. 1, Москва', telephone='8 (800) 200-54-34', lat=55.570166, long=37.575693), distance=6.104693355622902)]

Екатерина +7 916 138-17-11 55.871384 37.485332 [Neighbour(rosbank=Rosbank(adress='Смольная ул., 22, стр. 1, Москва', telephone='8 (800) 200-54-34', lat=55.860378, long=37.484379), distance=1.225255090163928)]

Елена +7 916 785-57-21 55.775635 37.684586 [Neighbour(rosbank=Rosbank(adress='Русаковская ул., 22, Москва', telephone='8 (800) 200-54-34'

In [55]:
pickle.dump(flat_owner_result, open('flat_owners_with_neighbour_v1.data', 'wb'))

In [9]:
data = pickle.load(open('flat_owners_with_neighbour_v1.data', 'rb'))

In [ ]:
Монолитных 23 странцы;
Кирпичных 25 страниц;
Панельных 25 страниц;  1-5 - 572, 6-10 - 493 
Монолитно-кирпичный 9 страниц.

# Визуализация данных

In [14]:
class FlatOwner:
    "Объект хозяин квартиры"
    
    def __init__(self, name, telephone, lat, long, neighbours, price, info):
        self.name = name
        self.telephone = telephone
        self.lat = lat # широта объекта недвижимости
        self.long = long # долгота объекта недвижимости
        self.neighbours = neighbours or list() # ближайшие отделения
        self.price = price or None # цена объекта недвижимости
        self.info = info or list() # площадь, жилая площадь, кухня, этаж, год постройки
        
    def __repr__(self):
        return '{}, {}, {}, тел. {}'.format(self.name, self.price,
                                    self.info[0], self.telephone)  
        #return '{} {} {} {}'.format(self.name, self.telephone,
                                           # self.lat, self.long)  
    
Rosbank = namedtuple('Rosbank', 'adress telephone lat long')

Neighbour = namedtuple('Neighbour', 'rosbank distance')
      
data = pickle.load(open('flat_owners_with_neighbour_v1.data', 'rb'))
rosbank_divisions = pickle.load(open('Rosbank_divisions_v1.data','rb'))

In [ ]:
for lat, long, name, price, adress in zip(latitudes, longitudes, names, price, adress):
    folium.CircleMarker((lat, long),
                    radius=5,
                    popup='{} {} \n Росбанк: {}'.format(name, price, adress),
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map)

In [16]:
map = folium.Map(location=(55.751244, 37.618423), zoom_start=12)

names =  [owner.name for owner in data]
latitudes =  [owner.lat for owner in data]
longitudes = [owner.long for owner in data]
price = [owner.price for owner in data]
adress = [owner.neighbours[0].rosbank.adress for owner in data]

for lat, long, owner, adress in zip(latitudes, longitudes, data, adress):
    folium.CircleMarker((lat, long),
                    radius=5,
                    popup='{}, Росбанк: {}'.format(owner, adress),
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map)

ros_latitudes = [bank.lat for bank in rosbank_divisions]
ros_longitudes = [bank.long for bank in rosbank_divisions]
ros_adress = [bank.adress for bank in rosbank_divisions]

for lat, long, adress in zip(ros_latitudes, ros_longitudes, ros_adress):
    folium.Marker((lat, long), popup='Росбанк '+str(adress), 
                  icon=folium.Icon(color='red',icon='bank', prefix='fa') ).add_to(map)
map

In [17]:
map.save(outfile='Moscow_map.html')